<a href="https://colab.research.google.com/github/sasansharifipour/EDA/blob/main/ImageNet_VGG16_FineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
if not os.path.exists('/root/.kaggle/'):
    os.makedirs('/root/.kaggle/')

file_download_link = "https://drive.google.com/u/0/uc?id=1YiIXXU1E1M7hHQsETKGaqY5H3uMLzX5r&export=download" 
!wget -O /root/.kaggle/kaggle.json --no-check-certificate "$file_download_link"

--2021-04-17 09:43:33--  https://drive.google.com/u/0/uc?id=1YiIXXU1E1M7hHQsETKGaqY5H3uMLzX5r&export=download
Resolving drive.google.com (drive.google.com)... 74.125.142.138, 74.125.142.100, 74.125.142.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.142.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-70-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ep09mrhdp4h136o2os45erne6u234cl1/1618652550000/14577798666193718332/*/1YiIXXU1E1M7hHQsETKGaqY5H3uMLzX5r?e=download [following]
--2021-04-17 09:43:33--  https://doc-04-70-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ep09mrhdp4h136o2os45erne6u234cl1/1618652550000/14577798666193718332/*/1YiIXXU1E1M7hHQsETKGaqY5H3uMLzX5r?e=download
Resolving doc-04-70-docs.googleusercontent.com (doc-04-70-docs.googleusercontent.com)... 74.125.20.132, 2607:f8b0:400e:c07::84
Connecting to doc-04-70-docs.googleusercontent.com (doc

In [2]:
!kaggle datasets download -d prithwirajmitra/covid-face-mask-detection-dataset

 96% 198M/207M [00:02<00:00, 96.1MB/s]
100% 207M/207M [00:02<00:00, 86.2MB/s]


In [3]:
!unzip covid-face-mask-detection-dataset.zip

Archive:  covid-face-mask-detection-dataset.zip
  inflating: New Masks Dataset/Test/Mask/2070.jpg  
  inflating: New Masks Dataset/Test/Mask/2072.jpg  
  inflating: New Masks Dataset/Test/Mask/2083.jpg  
  inflating: New Masks Dataset/Test/Mask/2085.jpg  
  inflating: New Masks Dataset/Test/Mask/2086.jpg  
  inflating: New Masks Dataset/Test/Mask/2110.jpg  
  inflating: New Masks Dataset/Test/Mask/2113.jpg  
  inflating: New Masks Dataset/Test/Mask/2114.jpeg  
  inflating: New Masks Dataset/Test/Mask/2130.jpg  
  inflating: New Masks Dataset/Test/Mask/2132.jpg  
  inflating: New Masks Dataset/Test/Mask/2135.jpg  
  inflating: New Masks Dataset/Test/Mask/2154.png  
  inflating: New Masks Dataset/Test/Mask/2158.png  
  inflating: New Masks Dataset/Test/Mask/2159.png  
  inflating: New Masks Dataset/Test/Mask/2160.png  
  inflating: New Masks Dataset/Test/Mask/2170.png  
  inflating: New Masks Dataset/Test/Mask/2172.png  
  inflating: New Masks Dataset/Test/Mask/2173.png  
  inflating: Ne

In [14]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range=0.2, horizontal_flip=True)

training_set = train_datagen.flow_from_directory("New Masks Dataset/Train/", target_size=(224,224),
                                                 batch_size=32, class_mode='categorical')

validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_set = validation_datagen.flow_from_directory('New Masks Dataset/Validation/', target_size = (224,224), 
                                                        batch_size = 32, class_mode = 'categorical')

test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory('New Masks Dataset/Test/', target_size = (224,224), 
                                                        batch_size = 32, class_mode = 'categorical')

Found 600 images belonging to 2 classes.
Found 306 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [10]:
from keras.applications import VGG16

vgg16_model = VGG16(weights="imagenet")

In [11]:
from keras.models import Sequential
from keras.layers import Activation, Dense

model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

for layer in model.layers:
    layer.trainable = False

model.add(Dense(units=2, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [12]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(x=training_set, epochs = 10, validation_data = validation_set, verbose=1)

Epoch 1/10
19/19 [==============================] - 16s 846ms/step - loss: 0.8013 - accuracy: 0.4657 - val_loss: 0.7243 - val_accuracy: 0.4869
Epoch 2/10
19/19 [==============================] - 16s 827ms/step - loss: 0.7045 - accuracy: 0.5432 - val_loss: 0.6863 - val_accuracy: 0.5556
Epoch 3/10
19/19 [==============================] - 15s 817ms/step - loss: 0.6509 - accuracy: 0.6369 - val_loss: 0.6341 - val_accuracy: 0.6471
Epoch 4/10
19/19 [==============================] - 15s 821ms/step - loss: 0.6017 - accuracy: 0.7261 - val_loss: 0.5969 - val_accuracy: 0.7288
Epoch 5/10
19/19 [==============================] - 15s 823ms/step - loss: 0.5553 - accuracy: 0.7589 - val_loss: 0.5625 - val_accuracy: 0.7582
Epoch 6/10
19/19 [==============================] - 15s 821ms/step - loss: 0.5432 - accuracy: 0.7444 - val_loss: 0.5277 - val_accuracy: 0.8464
Epoch 7/10
19/19 [==============================] - 15s 817ms/step - loss: 0.4930 - accuracy: 0.8257 - val_loss: 0.5066 - val_accuracy: 0.8301

In [17]:
loss, accuracy = model.evaluate(test_set, verbose=1)

4/4 [==============================] - 1s 242ms/step - loss: 0.4659 - accuracy: 0.8800


In [18]:
accuracy

0.8799999952316284